In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path = [
    p for p in sys.path
    if p not in ['/home/jxm3/research/prompting/imodelsX', '/home/jxm3/research/prompting/tree-prompt']
]
sys.path.append('/home/jxm3/research/retrieval/inversion')

In [13]:
import aliases

os.environ["TOKENIZERS_MULTIPROCESSING"] = "True"

# inv_trainer = aliases.load_trainer_from_alias("openai_msmarco__msl128__100epoch")
corr_experiment, corr_trainer = aliases.load_experiment_and_trainer_from_alias(
    "openai_msmarco__msl128__100epoch__correct"
)
inv_trainer = corr_trainer.inversion_trainer
# corr_trainer.precompute_hypotheses()
corr_trainer.model.eval()
print()

loading alias openai_msmarco__msl128__100epoch__correct from /home/jxm3/research/retrieval/inversion/saves/d6ec9d5838a4ad3daeba636e5378a8a0...
> checkpoint: /home/jxm3/research/retrieval/inversion/saves/d6ec9d5838a4ad3daeba636e5378a8a0/checkpoint-384000
Corrector encoder noise level 0.001
loading alias openai_msmarco__msl128__100epoch from /home/jxm3/research/retrieval/inversion/saves/f9abd65db4c4823264b133816d08612f...
Overwriting max sequence length from 128 to 128
Overwriting use_less_data from 1000000 to 1000000
> checkpoint: /home/jxm3/research/retrieval/inversion/saves/f9abd65db4c4823264b133816d08612f/checkpoint-194000
Loading datasets with TOKENIZERS_PARALLELISM = False


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output shape ->  torch.Size([1, 86])
	Decoded output -> And the trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, was brilliant, and smooty.
================ End trainer sanity check ================
Froze 353779584 params from model type <class 'models.inversion.InversionModel'>


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
(hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')
	Decoded output shape ->  torch.Size([1, 64])
	Decoded output -> Was brilliant, and witty, And witty, and witty, and witty, and witty, and witty, and witty, witty, witty, witty, witty, witty, witty.
================ End trainer sanity check ================



In [21]:
corr_trainer.return_best_hypothesis = False
corr_trainer.args.per_device_eval_batch_size = 128
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
corr_trainer.num_gen_recursive_steps = 5
metrics = corr_trainer.evaluate(
    # eval_dataset=corr_trainer.train_dataset.select(range(100)),
    eval_dataset=corr_trainer.eval_dataset["msmarco"].select(range(100)),
    metric_key_prefix="eval_test",
)
print(metrics)

[1] (hypothesis_input_ids == 1).sum(): tensor(85, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')


generating from val:   0%|                                                                                            | 0/1 [00:00<?, ?it/s]

[1] (hypothesis_input_ids == 1).sum(): tensor(85, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')


[pred] *The proposed parking lot will require a separate floor plan for each existing. parking space. [Submitting Use Permit: The Use Permit will show the dimensions of the drive space, total floor space, and total size of the drive space as shown on the.
[true] *The use of the driveway as the required parking spaces will require a separate submittal for a Use Permit. [ ] Floor plan (dimensioned): Show overall building size, existing and proposed room locations, existing and. proposed new window(s)/door(s) size/location(s).



[pred] Our INSPIRED Seville map is fully illuminated and enables you to explore each street in an effortless manner. Our hidden map of Seville includes a map of the streets of the city, a map of the streets of the city centre, and a map of the streets of the city's central business district.elcome to the streets of Seville, Costa Rica, Cartagena and Madrid, allowing you to explore the most coveted destinations with a distinctly rounded map of the city's central b

generating from val:   0%|                                                                                            | 0/1 [00:00<?, ?it/s]

[1] (hypothesis_input_ids == 1).sum(): tensor(85, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')


[pred] *The proposed parking lot will require a separate floor plan for each existing. parking space. [Submitting Use Permit: The Use Permit will show the dimensions of the drive space, total floor space, and total size of the drive space as shown on the.
[true] *The use of the driveway as the required parking spaces will require a separate submittal for a Use Permit. [ ] Floor plan (dimensioned): Show overall building size, existing and proposed room locations, existing and. proposed new window(s)/door(s) size/location(s).



[pred] Our INSPIRED Seville map is fully illuminated and enables you to explore each street in an effortless manner. Our hidden map of Seville includes a map of the streets of the city, a map of the streets of the city centre, and a map of the streets of the city's central business district.elcome to the streets of Seville, Costa Rica, Cartagena and Madrid, allowing you to explore the most coveted destinations with a distinctly rounded map of the city's central b

In [22]:
corr_trainer.return_best_hypothesis = False
corr_trainer.args.per_device_eval_batch_size = 128
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
corr_trainer.num_gen_recursive_steps = 5
metrics = corr_trainer.evaluate(
    # eval_dataset=corr_trainer.train_dataset.select(range(100)),
    eval_dataset=corr_trainer.eval_dataset["msmarco"].select(range(100)),
    metric_key_prefix="eval_test",
)
print(metrics)

[1] (hypothesis_input_ids == 1).sum(): tensor(85, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')


generating from val:   0%|                                                                                            | 0/1 [00:00<?, ?it/s]

[1] (hypothesis_input_ids == 1).sum(): tensor(85, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')
[1] (hypothesis_input_ids == 1).sum(): tensor(89, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')
[1] (hypothesis_input_ids == 1).sum(): tensor(89, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')
[1] (hypothesis_input_ids == 1).sum(): tensor(89, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')
[1] (hypothesis_input_ids == 1).sum(): tensor(85, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')


[pred] *The proposed use of the parking garage will require a separate floor plan for the existing. building. [Submit Submit Submit: Floor Plan Dimensions (shown on the permit)]: The total dimensions of all rooms, sizes, and location of driveways and entrances.
[true] *The use of the driveway as the required parking spaces will require a separate submittal for a Use Permit. [ ] Floor plan (dimensioned): Show overall building size, existing and proposed room locations, existing and. proposed new window(s)/door(s) size/location(s).



[pred] Our WEST SEVILLE street map is illuminated with a clearly marked entryway, allowing you to explore the city in an intuitive and accessible manner. Each map also includes a map of the city's hidden gems, a map of the city's quaint streets, and a map of the city's quaint hotels, restaurants and attractions.ur Costa Blanca hotels and apartments are a must-have destination for travelers of all ages and abilities.
[true] Our STREETWISE Seville Map enables

generating from val:   0%|                                                                                            | 0/1 [00:00<?, ?it/s]

[1] (hypothesis_input_ids == 1).sum(): tensor(85, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')
[1] (hypothesis_input_ids == 1).sum(): tensor(89, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')
[1] (hypothesis_input_ids == 1).sum(): tensor(89, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')
[1] (hypothesis_input_ids == 1).sum(): tensor(88, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')
[1] (hypothesis_input_ids == 1).sum(): tensor(86, device='cuda:0')
[2] (hypothesis_input_ids == 1).sum(): tensor(0, device='cuda:0')


[pred] *The proposed use of the parking garage will require a separate floor plan for the existing. building. [Submit Submit Submit: Floor Plan Dimensions (shown on the permit)]: The total dimensions of all rooms, sizes, and location of driveways and entrances.
[true] *The use of the driveway as the required parking spaces will require a separate submittal for a Use Permit. [ ] Floor plan (dimensioned): Show overall building size, existing and proposed room locations, existing and. proposed new window(s)/door(s) size/location(s).



[pred] Our WEST SEVILLE street map is illuminated with a clearly marked entryway, allowing you to explore the city in an intuitive and accessible manner. Each map also includes a map of the city's hidden gems, a map of the city's quaint streets, and a map of the city's quaint hotels, restaurants and attractions.ur Costa Blanca hotels and apartments are a must-have destination for travelers of all ages and abilities.
[true] Our STREETWISE Seville Map enables

In [ ]:
corr_trainer.return_best_hypothesis = False
corr_trainer.args.per_device_eval_batch_size = 128
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
corr_trainer.num_gen_recursive_steps = 1
metrics = corr_trainer.evaluate(
    # eval_dataset=corr_trainer.train_dataset.select(range(100)),
    eval_dataset=corr_trainer.eval_dataset["msmarco"].select(range(100)),
    metric_key_prefix="eval_test",
)
print(metrics)

In [ ]:
corr_trainer.return_best_hypothesis = False
corr_trainer.args.per_device_eval_batch_size = 128
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
corr_trainer.num_gen_recursive_steps = 1
metrics = corr_trainer.evaluate(
    eval_dataset=corr_trainer.train_dataset.select(range(500)),
    # eval_dataset=corr_trainer.eval_dataset["msmarco"].select(range(100)),
    metric_key_prefix="eval_test",
)
print(metrics)

In [12]:
print(metrics)

{'eval_test_loss': 1.146443486213684, 'eval_test_pred_num_tokens': 82.609375, 'eval_test_true_num_tokens': 80.90625, 'eval_test_token_set_precision': 0.60819345096195, 'eval_test_token_set_recall': 0.6866442994522647, 'eval_test_token_set_f1': 0.6400445672754415, 'eval_test_n_ngrams_match_1': 39.074, 'eval_test_n_ngrams_match_2': 20.53, 'eval_test_n_ngrams_match_3': 13.252, 'eval_test_num_true_words': 63.416, 'eval_test_num_pred_words': 63.594, 'eval_test_bleu_score': 29.071289956395393, 'eval_test_meteor_score': 0.5343961207296394, 'eval_test_rouge_score': 0.6376807383959213, 'eval_test_exact_match': 0.03, 'eval_test_emb_cos_sim': 0.9655262231826782, 'eval_test_runtime': 44.6905, 'eval_test_samples_per_second': 11.188, 'eval_test_steps_per_second': 0.09}
